<a href="https://colab.research.google.com/github/LeoCunha13/Imersao_Alura/blob/main/Converte_OFX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [172]:
import uuid
from datetime import datetime, timedelta
import re

def gerar_ofx(nome_arquivo_txt):
    """Converte um arquivo TXT com informações de extrato em um arquivo OFX.

    Args:
        nome_arquivo_txt (str): O nome do arquivo TXT contendo as informações do extrato.

    Returns:
        str: O conteúdo do arquivo OFX.
    """

    # Lê o conteúdo do arquivo TXT, especificando a codificação.
    with open(nome_arquivo_txt, "r", encoding="ISO-8859-1") as f:  # Use a codificação correta aqui!
        texto = f.read()

    # Define a data de "Ontem" como o dia anterior à data atual.
    ontem = datetime.now() - timedelta(days=1)
    ontem_str = ontem.strftime("%Y%m%d")

    # Extrai as informações relevantes do texto.
    transacoes = extrair_transacoes(texto, ontem)

    # Define as datas de início e fim do extrato.
    dtstart = min(t['DTPOSTED'] for t in transacoes) if transacoes else ontem_str + "000000[-3:GMT]"
    dtend = max(t['DTPOSTED'] for t in transacoes) if transacoes else ontem_str + "000000[-3:GMT]"

    # Cria o conteúdo do arquivo OFX.
    ofx = f"""OFXHEADER:100
            DATA:OFXSGML
            VERSION:102
            SECURITY:NONE
            ENCODING:USASCII
            CHARSET:1252
            COMPRESSION:NONE
            OLDFILEUID:NONE
            NEWFILEUID:NONE
            <OFX>
            <SIGNONMSGSRSV1>
            <SONRS>
            <STATUS>
            <CODE>0
            <SEVERITY>INFO
            </STATUS>
            <DTSERVER>{ontem}000000[-3:GMT]
            <LANGUAGE>POR
            </SONRS>
            </SIGNONMSGSRSV1>
            <CREDITCARDMSGSRSV1>
            <CCSTMTTRNRS>
            <TRNUID>1001
            <STATUS>
            <CODE>0
            <SEVERITY>INFO
            </STATUS>
            <CCSTMTRS>
            <CURDEF>BRL
            <CCACCTFROM>
            <ACCTID>5ca617c7-d250-4ce9-ae0a-d3126907d00b
            </CCACCTFROM>
            <BANKTRANLIST>
            <DTSTART>{dtstart}
            <DTEND>{dtend}
            """

    for transacao in transacoes:
        ofx += f"""
              <STMTTRN>
              <TRNTYPE>{transacao['TRNTYPE']}
              <DTPOSTED>{transacao['DTPOSTED']}
              <TRNAMT>{transacao['TRNAMT']}
              <FITID>{transacao['FITID']}
              <MEMO>{transacao['MEMO']}
              </STMTTRN>
              """

    return ofx




In [192]:
import uuid
from datetime import datetime, timedelta
import re

def extrair_transacoes(texto, ontem):
    """Extrai as transações do texto, considerando a estrutura fornecida.

    Args:
        texto (str): O texto contendo as informações do extrato.
        ontem (datetime): A data de referência para "Ontem".

    Returns:
        list: Uma lista de dicionários representando as transações.
    """

    transacoes = []

    texto = texto.strip()
    linhas = [linha for linha in texto.split("\n") if linha.strip() != ""]

    dias_da_semana = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado", "Domingo"]
    meses = {"MAI": 5, "JUN": 6, "JUL": 7, "AGO": 8, "SET": 9, "OUT": 10, "NOV": 11, "DEZ": 12}

    dias_map = {dia: (ontem - timedelta(days=(ontem.weekday() - i) % 7)).strftime("%Y%m%d")
                 for i, dia in enumerate(dias_da_semana)}

    dia_atual = None
    mes_atual = None
    for linha in linhas:
        # Ignora as linhas contendo "Saldo em atraso" e "Crédito de atraso".
        if "Saldo em atraso" in linha or "Crédito de atraso" in linha:
            continue

        # Verifica se a linha contém um dia da semana.
        for dia in dias_da_semana:
            if linha.startswith(dia):
                dia_atual = dias_map[dia]
                break

        # Verifica se a linha contém um mês abreviado.
        for mes_abrev, mes_num in meses.items():
            if mes_abrev in linha:
                mes_atual = mes_num
                partes = linha.split(mes_abrev)
                dia_atual = int(partes[0].strip())
                ano_atual = ontem.year  # Considera o ano atual como referência
                dia_atual = datetime(ano_atual, mes_atual, dia_atual).strftime("%Y%m%d")
                break

        # Se encontrou um dia ou um mês, tenta extrair a transação.

        if dia_atual:
            partes = linha.split("R$")
            print(partes[0].strip())
            descricao = partes[0].strip()
            valor = float(partes[1].strip().replace(".", "").replace(",", "."))
            transacoes.append({
                    "TRNTYPE": "DEBIT",
                    "DTPOSTED": f"{dia_atual}000000[-3:GMT]",
                    "TRNAMT": f"{-valor:.2f}",
                    "FITID": "",
                    "MEMO": descricao
                })

    return transacoes

In [193]:
# ... (código das funções)

# Define o nome do arquivo TXT.
nome_arquivo_txt = "extrato.txt"

# Chama a função gerar_ofx() com o nome do arquivo TXT.
conteudo_ofx = gerar_ofx(nome_arquivo_txt)

# Define o nome do arquivo OFX de saída.
nome_arquivo_ofx = "extrato_formatado.ofx"

# Salva o conteúdo em um arquivo OFX.
with open(nome_arquivo_ofx, "w") as f:
    f.write(conteudo_ofx)

print(f"Arquivo OFX gerado com sucesso: {nome_arquivo_ofx}")

2 JUN


IndexError: list index out of range